In [23]:
%%capture
%pip install pulp

In [24]:
from pulp import *

## Dados reais de algumas das modalidades mais comuns

In [25]:
CLIENTES = [
	{
		'modalidade': 'Capital de giro',
		'propostas_recebidas': 2051,
		'taxa_inadimplencia': 0.17,
		'taxa_juros': 2.13,
	},
	{
		'modalidade': 'Cheque Especial',
		'propostas_recebidas': 2242,
		'taxa_inadimplencia': 0.60,
		'taxa_juros': 8.16,
	},
	{
		'modalidade': 'Crédito Pessoal',
		'propostas_recebidas': 1982,
		'taxa_inadimplencia': 0.36,
		'taxa_juros': 6.43,
	},
	{
		'modalidade': 'Crédito Pessoal Consignado',
		'propostas_recebidas': 218,
		'taxa_inadimplencia': 0.00,
		'taxa_juros': 2.36,
	},
	{
		'modalidade': 'Financiamento Imobiliário',
		'propostas_recebidas': 1802,
		'taxa_inadimplencia': 0.08,
		'taxa_juros': 0.81,
	},
	{
		'modalidade': 'Aquisição de veículos',
		'propostas_recebidas': 1802,
		'taxa_inadimplencia': 0.12,
		'taxa_juros': 1.86,
	},
]

## Definição das variáveis

In [26]:
problema = LpProblem('Maximizar_Lucro_Implementacao_Gradual', LpMaximize)

CapitalDeGiro = LpVariable('Capital_de_giro', lowBound=0)
ChequeEspecial = LpVariable('Cheque_Especial', lowBound=0)
CreditoPessoal = LpVariable('Crédito_Pessoal', lowBound=0)
CreditoPessoalConsignado = LpVariable('Crédito_Pessoal_Consignado', lowBound=0)
FinanciamentoImobiliario = LpVariable('Financiamento_Imobiliário', lowBound=0)
AquisicaoDeVeiculos = LpVariable('Aquisição_de_veículos', lowBound=0)

variaveis = {
	'Capital de giro': CapitalDeGiro,
	'Cheque Especial': ChequeEspecial,
	'Crédito Pessoal': CreditoPessoal,
	'Crédito Pessoal Consignado': CreditoPessoalConsignado,
	'Financiamento Imobiliário': FinanciamentoImobiliario,
	'Aquisição de veículos': AquisicaoDeVeiculos,
}

#### Calculando o lucro gerado por cada modalidade a partir da quantidade escolhida de clientes

In [27]:
problema += lpSum(
	((1 - C['taxa_inadimplencia']) * C['taxa_juros'] * variaveis[C['modalidade']])
	for C in CLIENTES
)

## Restrições

#### Restringindo por quantidade máxima de clientes

Quantos clientes vamos suportar no primeiro trimestre da nossa implementação gradual?

In [28]:
MAX_CLIENTES_TRIMESTRE = 6000

In [29]:
problema += lpSum(variaveis.values()) <= MAX_CLIENTES_TRIMESTRE

#### Restringindo por total de número de clientes interessados

In [30]:
for i in range(len(CLIENTES)):
	C = CLIENTES[i]
	V = variaveis[C['modalidade']]
	problema += V <= C['propostas_recebidas']

#### Restringindo por número mínimo de clientes que precisamos pegar de cada modalidade

Vamos pegar pelo menos um pouco de cada modalidade, assim podemos ter uma ideia de como a nossa solução vai se comportar

In [31]:
AMOSTRA_MINIMA_POSSIVEL = 400

In [32]:
for V in variaveis.values():
	problema += V >= AMOSTRA_MINIMA_POSSIVEL

#### Restringindo a quantidade de clientes de Cheque Especial, para evitar tantos problemas logo no começo

In [33]:
problema += ChequeEspecial <= 1000

## Resolvendo a otimização

In [34]:
problema.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/consultor/.asdf/installs/python/3.12.7/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b0cb962d42be4b0aa2c5c71229c4ac74-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/b0cb962d42be4b0aa2c5c71229c4ac74-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 19 COLUMNS
At line 45 RHS
At line 60 BOUNDS
At line 61 ENDATA
Problem MODEL has 14 rows, 6 columns and 19 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj -0 Primal inf 2400 (6) Dual inf 13.889099 (6)
0  Obj -0 Primal inf 2400 (6) Dual inf 6e+10 (6)
7  Obj 16531.169 Primal inf 182 (1)
Primal infeasible - objective value 16531.169
PrimalInfeasible objective 16531.1686 - 7 iterations time 0.002

Re

-1

## Visualização dos resultados

#### Quantidade de clientes total:

In [35]:
for C in CLIENTES:
    print(f'{C["modalidade"]}: {C["propostas_recebidas"]}')

Capital de giro: 2051
Cheque Especial: 2242
Crédito Pessoal: 1982
Crédito Pessoal Consignado: 218
Financiamento Imobiliário: 1802
Aquisição de veículos: 1802


#### Quantidade de clientes escolhida:

In [36]:
for C in CLIENTES:
    print(f"{C['modalidade']}: {int(variaveis[C['modalidade']].varValue)}")

Capital de giro: 1818
Cheque Especial: 1000
Crédito Pessoal: 1982
Crédito Pessoal Consignado: 400
Financiamento Imobiliário: 400
Aquisição de veículos: 400
